In [1]:
import pandas as pd
import numpy as np
import torch.nn as nn
import torch.optim as optim

In [4]:
mistral_df = pd.read_csv("mistral_data.csv")  # 0-AI, 1-HUMAN
persuade_df = pd.read_csv("persuade.csv")  # ALL HUMAN
gpt_df = pd.read_csv("chat_gpt_essay.csv")  # ALL AI
comp_df = pd.read_csv("train_essays_comp.csv")  # 1-AI, 0-HUMAN
daigt_df = pd.read_csv("daigt_train.csv")  # 1-AI, 0-HUMAN

human_text_mistral = mistral_df.query('LABEL == 1')['Text']
human_text_persuade = persuade_df['full_text'].sample(frac=1)
human_text_comp = comp_df.query('generated == 0')['text']
human_text_daigt = daigt_df.query('label == 0')['text']

ai_text_mistral = mistral_df.query('LABEL == 0')['Text']
ai_text_gpt = gpt_df['Text']
ai_text_comp = comp_df.query('generated == 1')['text']
ai_text_daigt = daigt_df.query('label == 1')['text']

human_texts = pd.concat([human_text_mistral, human_text_comp, human_text_daigt], ignore_index=True)
ai_texts = pd.concat([ai_text_mistral, ai_text_gpt, ai_text_comp, ai_text_daigt], ignore_index=True)

df_human = pd.DataFrame(human_texts, columns=['text'])
df_human['generated'] = 0 

df_ai = pd.DataFrame(ai_texts, columns=['text'])
df_ai['generated'] = 1  

df_human = df_human.iloc[:20000] #only 20k
df_ai = df_ai.iloc[:20000]

df_final = pd.DataFrame(columns = ['text', 'generated'])
df_final = pd.concat([df_human, df_ai, df_final])
df_final = df_final.sample(frac = 1)

In [5]:
df_final

,text,generated
4907,"Dear [Principal’s Name],\n\nI am writing this ...",1
12531,"""Making Mona Lisa Smile"" is about a computer h...",0
11359,The author supports the idea that studying Ven...,0
6070,"In my opinion, it is not a good idea to gradu...",1
18939,Driverless cars will be available in the futur...,0
...,...,...
8761,Do you ever want to do something fun at your s...,0
17063,"The sky was a clear blue, and the sun was shin...",1
12108,In this article it talks about the face of Mon...,0
3673,"Hey there! So, you wanna know about the benef...",1


In [6]:
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab, build_vocab_from_iterator
from torch.utils.data import Dataset
import torch

tokenizer = get_tokenizer('basic_english')

def build_vocab(texts):
    for text in texts:
        yield tokenizer(text)

vocab_df = Vocab(build_vocab_from_iterator(build_vocab(df_final['text']), max_tokens = 20000, min_freq = 1, specials=('<unk>', '<pad>')))
vocab_df.set_default_index(vocab_df['<unk>'])

In [7]:
for token in tokenizer("Hello this is Mridul"):
    print(vocab_df[token])

3819
24
12
0


In [8]:
import torchtext.vocab as vocab

glove = vocab.GloVe(name='6B', dim=100)

In [9]:
def get_glove_embedding(vocab_df, glove):
    emb_dim = glove.dim
    embedding = torch.zeros(vocab_df.__len__(), emb_dim)
    num_loaded = 0
    
    for i, token in enumerate(vocab_df.get_itos()):
        if token in glove.stoi:
            embedding[i] = glove[token]
            num_loaded += 1
        else:
            embedding[i] = 0 #torch.randn(emb_dim)
    
    print(f"Loaded {num_loaded} words out of {len(vocab_df)} total")
    return embedding

embedding_weight = get_glove_embedding(vocab_df, glove)

Loaded 16640 words out of 20000 total


In [10]:
from tqdm import tqdm 

def text_to_tensor(texts, vocab_df, max_len=200):
    tensor = torch.ones((len(texts), max_len), dtype=torch.long)
    
    for i, text in tqdm(enumerate(texts)):
        tokenized_text = tokenizer(text)
        for j, token in enumerate(tokenized_text):
            if j >= max_len:
                break
                
            try:
                tensor[i, j] = vocab_df.__getitem__(token)
            except:
                tensor[i, j] = 0
            #tensor[i, j] = vocab_df.get_stoi().get(token, vocab_df['<unk>'])
    
    return tensor

text_tensors = text_to_tensor(df_final['text'], vocab_df)

39300it [00:32, 1204.00it/s]


In [11]:
labels = np.array(df_final['generated'], dtype = np.int64)

In [12]:
from torch.utils.data import DataLoader, Dataset
import torch

class TextDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

train_sent = text_tensors[:38000]
train_labels = labels[:38000]
test_sent = text_tensors[38000:]
test_labels = labels[38000:]
    
train_dataset = TextDataset(train_sent, train_labels)
train_loader = DataLoader(dataset=train_dataset, batch_size=32, shuffle=True)

test_dataset = TextDataset(test_sent, test_labels)
test_loader = DataLoader(dataset=test_dataset, batch_size=32, shuffle=True)

In [15]:
import torch
import torch.nn as nn

class IdentifyText(nn.Module):
    def __init__(self, embedding_weights, 
                 hidden_dim, output_dim=2, 
                 num_layers=2, dropout=0.2):
        super(IdentifyText, self).__init__()
        
        self.embedding = nn.Embedding.from_pretrained(embedding_weights, freeze=True)
        
        # Ensuring dropout is only applied if there are multiple LSTM layers
        effective_dropout = dropout if num_layers > 1 else 0
        self.lstm = nn.LSTM(embedding_weights.shape[1], hidden_dim, 
                            num_layers=num_layers, batch_first=True, 
                            dropout=effective_dropout)
        
        self.fully_connected_1 = nn.Linear(hidden_dim, 64)
        self.fully_connected_2 = nn.Linear(64, 64)
        self.fully_connected_3 = nn.Linear(64, 32)
        
        self.dropout = nn.Dropout(dropout)    
        self.softmax = nn.Softmax(dim = 1)
        self.out = nn.Linear(32, output_dim)
        
    def forward(self, text):
        embedded = self.embedding(text)  # [batch size, sent_length]
            
        output_lstm, (hidden_lstm, cell) = self.lstm(embedded)
        
        hidden_1 = torch.relu(self.fully_connected_1(hidden_lstm[-1]))
        hidden_2 = torch.relu(self.fully_connected_2(hidden_1))
        hidden_3 = torch.relu(self.fully_connected_3(hidden_2))
        
        output = self.softmax(self.out(hidden_3))
        
        return output

In [16]:
hidden_dim = 64
output_dim = 2  
num_layers = 2
dropout = 0.5

model = IdentifyText(embedding_weight, hidden_dim, output_dim, num_layers, dropout)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [17]:
def train_model(model, train_loader, optimizer, criterion, num_epochs=10):
    model.train()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for texts, labels in train_loader:
#             texts = texts.to(device) 
#             labels = labels.to(device).float()

            # Forward pass
            predictions = model(texts)
            loss = criterion(predictions, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')

In [18]:
train_model(model, train_loader, optimizer, criterion, num_epochs=10)

Epoch 1, Loss: 0.6720302944582721
Epoch 2, Loss: 0.5082429432206683
Epoch 3, Loss: 0.7543646959644376
Epoch 4, Loss: 0.8229418499802901
Epoch 5, Loss: 0.8229155460190692
Epoch 6, Loss: 0.8228629353873256
Epoch 7, Loss: 0.756515675973812
Epoch 8, Loss: 0.42742429415284583
Epoch 9, Loss: 0.3816421390934424
Epoch 10, Loss: 0.37804312374355015


In [19]:
test_sent = "Using phone while driving is not good."
encoded_test = text_to_tensor([test_sent], vocab_df)

print(encoded_test)

1it [00:00, 300.58it/s]

tensor([[207, 181,  75,  84,  12,  26,  93,   2,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
           1,   1,   1,   1,   1,   1,   1,   1,   1

In [24]:
_, pred = torch.max(model(encoded_test).data, 1)
print(pred)

tensor([1])


In [26]:
def test_model(model):
    model.eval() 
    correct = 0
    total = 0
    
    with torch.no_grad(): 
        for texts, labels in test_loader:
            texts = texts
            labels = labels
            outputs = model(texts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test data: {accuracy:.2f}%')
    return accuracy

test_model(model)

Accuracy of the model on the test data: 94.62%


94.61538461538461

# Paraphrase

In [29]:
para_data = pd.read_csv("parahrased_human.csv")
para_human = np.array(para_data['Paraphrased']) #output should be AI gen

revised_para_human = []
for string in para_human:
    if type(string) != float:
        revised_para_human.append(string)

revised_para_human = np.array(revised_para_human)
para_tensors = text_to_tensor(revised_para_human, vocab_df)

para_label = np.ones(97)

para_dataset = TextDataset(para_tensors, para_label)
para_loader = DataLoader(dataset=para_dataset, batch_size=32, shuffle=True)

97it [00:00, 646.22it/s]


In [30]:
def para_model(model):
    model.eval() 
    correct = 0
    total = 0
    
    with torch.no_grad(): 
        for texts, labels in para_loader:
            texts = texts
            labels = labels
            outputs = model(texts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test data: {accuracy:.2f}%')
    return accuracy

para_model(model)

Accuracy of the model on the test data: 96.91%


96.90721649484536

# CHATGPT

In [35]:
para_gpt = pd.read_csv("GPTparaphrased.csv")
para_gpt_text = np.array(para_gpt['paraphrased'])

para_gpt_label = np.ones(para_gpt['LABEL'])

para_gpt_tensors = text_to_tensor(para_gpt_text, vocab_df)

1500it [00:01, 1234.51it/s]


In [36]:
para_dataset_gpt = TextDataset(para_gpt_tensors, para_gpt_label)
para_loader_gpt = DataLoader(dataset=para_dataset_gpt, batch_size=32, shuffle=True)

In [37]:
def para_model_gpt(model):
    model.eval() 
    correct = 0
    total = 0
    
    with torch.no_grad(): 
        for texts, labels in para_loader_gpt:
            texts = texts
            labels = labels
            outputs = model(texts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test data: {accuracy:.2f}%')
    return accuracy

para_model_gpt(model)

Accuracy of the model on the test data: 99.47%


99.46666666666667

# Human

In [45]:
para_human_gpt = pd.read_csv("GPT4_HumanParaphrase.csv")
para_human_gpt_text = para_human_gpt['Text']

revised_para_gpt_human = []
for string in para_human_gpt_text:
    if type(string) != float:
        revised_para_gpt_human.append(string)
        
print(len(revised_para_gpt_human))


para_human_gpt_label = np.ones(len(revised_para_gpt_human))

1497


In [46]:
para_human_gpt_tensors = text_to_tensor(revised_para_gpt_human, vocab_df)

1497it [00:01, 1286.59it/s]


In [47]:
para_dataset_gpt_human = TextDataset(para_human_gpt_tensors, para_human_gpt_label)
para_loader_gpt_human = DataLoader(dataset=para_dataset_gpt_human, batch_size=32, shuffle=True)

In [49]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [50]:
def para_model_gpt_human(model):
    model.eval() 
    correct = 0
    total = 0
    
    all_predictions = []
    all_labels = []
    
    with torch.no_grad(): 
        for texts, labels in para_loader_gpt_human:
            texts = texts
            labels = labels
            outputs = model(texts)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            all_predictions.extend(predicted)
            all_labels.extend(labels)
            
    precision = precision_score(all_labels, all_predictions, average='binary')
    recall = recall_score(all_labels, all_predictions, average='binary')
    f1 = f1_score(all_labels, all_predictions, average='binary')

    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')

    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the test data: {accuracy:.2f}%')
    
    return accuracy

para_model_gpt_human(model)

Precision: 1.00
Recall: 0.90
F1 Score: 0.95
Accuracy of the model on the test data: 90.38%


90.38076152304609